In [5]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)  

In [6]:
# SELECT ELECTION YEAR (1976-2016)
election_year = 2016

In [7]:
df = pd.read_csv('1976-2016-president.csv')
electoral_votes = pd.read_csv('electoral_votes.csv')
state_list = list(df.state.unique())

In [8]:
df.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party,writein,candidatevotes,totalvotes,version,notes
0,1976,Alabama,AL,1,63,41,US President,"Carter, Jimmy",democrat,False,659170,1182850,20171015,NaN
1,1976,Alabama,AL,1,63,41,US President,"Ford, Gerald",republican,False,504070,1182850,20171015,NaN
2,1976,Alabama,AL,1,63,41,US President,"Maddox, Lester",american independent party,False,9198,1182850,20171015,NaN
3,1976,Alabama,AL,1,63,41,US President,"Bubar, Benjamin """"Ben""""",prohibition,False,6669,1182850,20171015,NaN
4,1976,Alabama,AL,1,63,41,US President,"Hall, Gus",communist party use,False,1954,1182850,20171015,NaN


In [9]:
election_results = df[['year','state','candidate','party','candidatevotes','totalvotes']]
election_results.head()

,year,state,candidate,party,candidatevotes,totalvotes
0,1976,Alabama,"Carter, Jimmy",democrat,659170,1182850
1,1976,Alabama,"Ford, Gerald",republican,504070,1182850
2,1976,Alabama,"Maddox, Lester",american independent party,9198,1182850
3,1976,Alabama,"Bubar, Benjamin """"Ben""""",prohibition,6669,1182850
4,1976,Alabama,"Hall, Gus",communist party use,1954,1182850


In [10]:
selected_election_results = election_results[(election_results['year'] == election_year) & 
                                             ((election_results['party'] == 'republican') | (election_results['party'] == 'democrat'))]
df_gen = []
for i, state in enumerate(state_list):
    evotes = int(electoral_votes[electoral_votes['state']==state].evotes)
    df_gen.append([state, 'republican',selected_election_results[(selected_election_results['state']==state) &
                                                                 (selected_election_results['party']=='republican')].candidatevotes.sum(), evotes])
    df_gen.append([state, 'democrat',selected_election_results[(selected_election_results['state']==state) & 
                                                               (selected_election_results['party']=='democrat')].candidatevotes.sum(), evotes])

In [11]:
clean_election_results = pd.DataFrame(df_gen, columns=['state','party','votes', 'evotes'])
clean_election_results

,state,party,votes,evotes
0,Alabama,republican,1318255,9
1,Alabama,democrat,729547,9
2,Alaska,republican,163387,3
3,Alaska,democrat,116454,3
4,Arizona,republican,1252401,11
5,Arizona,democrat,1161209,11
6,Arkansas,republican,684872,6
7,Arkansas,democrat,380494,6
8,California,republican,4483810,55
9,California,democrat,8753788,55
